15分でできる日本語Word2Vec https://qiita.com/makaishi2/items/63b7986f6da93dc55edd

In [1]:
import zipfile
import re
import json
import urllib.request
from janome.tokenizer import Tokenizer
from gensim.models import word2vec

C:\Users\haruk\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
# zipファイルダウンロード
url = 'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'
zip = 'data/794_ruby_4237.zip'
urllib.request.urlretrieve(url, zip)

# ダウンロードしたzipの解凍

with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    # 解凍後のファイルからデータ読み込み
    for myfile in myzip.infolist():
        # 解凍後ファイル名取得
        filename = myfile.filename
        # ファイルオープン時にencodingを指定してsjisの変換をする
        with open(filename, encoding='sjis') as file:
            text = file.read()

text = re.split('\-{5,}',text)[2] # ヘッダ部分の除去
text = re.split('底本：',text)[0] # フッタ部分の除去
text = text.replace('|', '') # | の除去
text = re.sub('《.+?》', '', text) # ルビの削除
text = re.sub('［＃.+?］', '',text) # 入力注の削除
text = re.sub('\n\n', '\n', text) # 空行の削除
text = re.sub('\r', '', text)
print(text[:100])


一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい


In [3]:
# Tokenizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

一
する
目
さめる
女
隣
じいさん
話
始める
いる


## 引数の確認

sentences=None, 
corpus_file=None, 
vector_size=100, 
alpha=0.025, 
window=5, 
min_count=5, 
max_vocab_size=None, 
sample=0.001, 
seed=1, 
workers=3, 
min_alpha=0.0001, 
sg=0, 
hs=0, 
negative=5, 
ns_exponent=0.75, 
cbow_mean=1, 
hashfxn=<built-in function hash>, 
epochs=5, 
null_word=0, 
trim_rule=None, 
sorted_vocab=1, 
batch_words=10000, 
compute_loss=False, 
callbacks=(), 
comment=None, 
max_final_vocab=None)

In [4]:
# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
# print(help(word2vec.Word2Vec()))
model = word2vec.Word2Vec(word_list, vector_size=100,min_count=5,window=5,epochs=100)

In [5]:
print(model.__dict__['wv']['世間'])

[ 1.47738531e-01  7.48094738e-01  1.64193794e-01 -5.00450991e-02
 -1.56591520e-01 -7.58504808e-01 -1.34193093e-01  5.31632841e-01
 -6.82547808e-01  1.91176414e-01  3.73182595e-01 -3.73062283e-01
  8.95477980e-02  7.97578990e-01  4.86545593e-01 -1.96622685e-03
 -2.85369724e-01  2.21585780e-01 -3.10054034e-01 -4.24489886e-01
 -1.00921047e+00  3.56334060e-01  6.56507313e-01 -1.15942442e+00
  1.91227272e-01 -3.03334653e-01 -7.99480319e-01 -5.03003299e-01
  1.05372846e-01 -5.54471970e-01  4.69458342e-01 -3.68871421e-01
 -1.75862461e-01 -4.91090566e-02 -4.32000726e-01  3.84856999e-01
 -6.09925985e-01 -4.05071348e-01 -3.96601111e-01 -2.61569619e-01
  5.17423213e-01  3.67114656e-02 -1.88329250e-01 -4.98350680e-01
 -5.22106409e-01  4.44581687e-01 -3.90957385e-01 -4.40871358e-01
  2.72994250e-01 -2.24807605e-01  8.15057397e-01 -4.93191123e-01
  4.78905648e-01 -4.44945037e-01  2.65440911e-01 -9.66878831e-01
  2.88865209e-01 -1.84856603e-04 -4.50716823e-01  3.17881644e-01
 -5.72665930e-01  6.90720

In [6]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['世間']) 
for item in ret:
    print(item[0], item[1])

聞こえる 0.5858389139175415
自己 0.5509515404701233
喝采 0.5320912003517151
決心 0.5112772583961487
堪える 0.4917698800563812
社会 0.480580598115921
賛成 0.47687000036239624
影響 0.46901530027389526
外国 0.44975388050079346
未来 0.4467791020870209


In [7]:
path = 'data/recipe_chefgohan.json'
df = json.load(open(path, 'r', encoding='utf-8'))
# print(df)
text = ''
for _,r in df['recipes'].items():
    # print(r['title'])
    start = r['title']+'を作ります。'
    end = r['title']+'が完成しました。'
    instructions = ''.join(r['instructions']).replace('\n','')
    text += start + instructions + end + '\n'
print(text[:100])

さつま芋のもちもちチーズ羊羹を作ります。牛乳と生クリームを合わせ、棒寒天を加えて煮溶かす。下準備したさつまいもとクリームチーズを合わせてよく混ぜてSTEP1をこしながら加える。さらに小麦粉とラム酒を加


In [8]:
# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

さつま芋
もち
もち
チーズ
羊羹
作る


In [9]:
model = word2vec.Word2Vec(word_list, vector_size=300,min_count=5,window=5,epochs=100)

In [10]:
print(model.__dict__['wv']['肉'])

[-0.86755484 -0.1890513   2.70156    -0.76835483  1.2123765  -1.7350413
  1.2958905  -1.2852497  -0.27418274  0.02897273 -1.6959829   1.7419792
 -0.9828316  -0.10096513 -0.08968378 -0.48344788  0.42379144 -0.6499651
 -0.23307022 -0.23208265  0.4792553   0.6551309   0.5196844  -0.59654915
 -1.0083596  -0.8953712  -1.5150737  -0.5737016   0.3320661  -1.1033993
 -0.9861647   1.3220124   0.575487    0.0034522   1.2450875  -0.17649202
  0.47227368  1.9448894   1.171233   -1.1876876  -0.02460204 -1.5420787
 -0.9351952   1.5278754   1.0701902   0.1521138   0.98993456  0.21089341
  0.4407267  -1.0350362   0.79633874  0.03769383 -0.13851017 -1.1739333
 -0.05199483  1.0044297   1.8392434  -0.6603198   1.4958344   0.10411313
  1.1236842   0.19617034  0.63595134 -0.08532842  0.5470934   0.4903468
 -0.5402613   1.3054796  -1.1065677  -1.2565951  -0.83286136 -1.0231458
  0.17098995 -0.22348914  0.16459581  0.6537293  -0.40904838  0.95208085
  0.88502413  0.01085455 -0.6184819  -0.16556258 -1.4195461

In [11]:
ret = model.wv.most_similar(positive=['焼く']) 
for item in ret:
    print(item[0], item[1])

焼ける 0.4931952655315399
キツネ 0.45121780037879944
トースター 0.44726747274398804
焼き上げる 0.430612713098526
返す 0.4264180660247803
両面 0.4043942987918854
面 0.4029857814311981
押す 0.3814873993396759
焼き 0.37664249539375305
シート 0.36231309175491333


In [12]:
results = model.wv.most_similar(positive=['焼く','水'], topn=5)
for result in results:
    print(result[0])

焦げる
g
70
焼き上げる
テフロン


In [13]:
results = model.wv.most_similar(positive=['煮る'], negative=['水'], topn=5)
for result in results:
    print(result[0])

煮込む
タイ
店
俺
蒸し器


In [14]:
results = model.wv.most_similar(positive=['煮る'], negative=['焼く'], topn=5)
for result in results:
    print(result[0])

顆粒
だし
A
煮汁
ペースト


In [15]:
results = model.wv.most_similar(positive=['肉','たまねぎ'], topn=5)
for result in results:
    print(result[0])

きゅうり
挽き肉
ひき肉
ローリエ
セロリ
